## 文本相似模型(Sentence Similarity)

### 載入相關套件 

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [2]:
datasets = load_dataset('roberthsu2003/for_Sentence_Similarity')
datasets

README.md:   0%|          | 0.00/455 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/271k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

### 數據集處理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
